#IMA206 PROJECT - Comprehension and exploration of a high-performance generative model: the StyleGAN - 

**Students**: 

*   Gabriela Barbosa Guedes Pereira

* Lucas Pereira Fernandes

* Lucas Oliveira Machado de Sousa

* Marina de Menezes Lima


**Advisors**: 
* Alasdair Newson
* Gwilherm Lesné


**June 2022 - Telecom Paris**


StyleGAN2-ADA-PyTorch 

**Notes**

This is based on Derrick Schultz's [SG2-ADA-PyTorch notebook](https://colab.research.google.com/github/dvschultz/stylegan2-ada-pytorch/blob/main/SG2_ADA_PyTorch.ipynb).

## Setup

Let’s start by checking to see what GPU we’ve been assigned.

In [ ]:
!nvidia-smi -L

Next let’s connect our Google Drive account.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install repo

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [ ]:
import os
!pip install gdown --upgrade

if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

## Dataset Preparation

Upload a .zip of square images to the `datasets` folder.

## Train model

Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `resume_from`: we've uploaded the "metfaces" file for network parameters.
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [ ]:
resume_from = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/celebahq-res256-mirror-paper256-kimg100000-ada-target0.5.pkl'
aug_strength = 0.0

In [ ]:
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/celebahq-res256-mirror-paper256-kimg100000-ada-target0.5.pkl
!mv celebahq-res256-mirror-paper256-kimg100000-ada-target0.5.pkl /content/


In [ ]:
%cd /content/
!git clone https://github.com/gtamba/pytorch-slim-cnn 

In [ ]:
%cd pytorch-slim-cnn/

In [ ]:
from slimnet import SlimNet

In [ ]:
%cd /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch

In [ ]:
import torch
import pickle
import PIL
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
import random

In [ ]:
device = torch.device('cuda')
model = SlimNet.load_pretrained('/content/pytorch-slim-cnn/models/celeba_20.pth').to(device)

In [ ]:
labels = np.array(['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
       'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
       'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
       'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young'])

In [ ]:
transform = transforms.Compose([
                              transforms.Resize((178,218)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

In [ ]:
with open('/content/celebahq-res256-mirror-paper256-kimg100000-ada-target0.5.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module

## Get images with/without glasses

This was executed once to identify multiple images with and without the presence of glasses. Then, we handpicked the good ones (since some of the images were rather badly generated) and saved these images and the points in latent space used to generate them into .npy files, so we don't have to run the code and handpick the images again every time.

In [ ]:
w_glasses = []
for cont in range(400):
  target = set([''])
  while 'Eyeglasses' not in target:
    n_imgs = 1
    z = torch.randn([n_imgs, G.z_dim]).cuda()    # latent codes
    c = None                                # class labels (not used in this example)]
    w = G.mapping(z, c, truncation_psi=1, truncation_cutoff=8)
    img = G.synthesis(w, noise_mode='const', force_fp32=True)

    img = (img.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    x = transform(PIL.Image.fromarray(img.cpu().numpy(), 'RGB')).unsqueeze(0).to(device)
    with torch.no_grad():
      model.eval()
      logits = model(x)
      sigmoid_logits = torch.sigmoid(logits)
      predictions = (sigmoid_logits > 0.5).squeeze().cpu().numpy()
    target = set(labels[predictions.astype(bool)])
  w_glasses.append(w)

In [ ]:
for i in range(400):
  img = G.synthesis(w_glasses[i], noise_mode='const', force_fp32=True)
  img = (img.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  img_print = np.array(PIL .Image.fromarray(img.cpu().numpy(), 'RGB'))
  plt.ion()
  plt.figure()
  plt.imshow(img_print)
  plt.title(f'{i}')
  plt.show()
  _ = input('')
  plt.close()

In [ ]:
w_noglasses = []
for cont in range(400):
  target = set(['Eyeglasses'])
  while 'Eyeglasses' in target:
    n_imgs = 1
    z = torch.randn([n_imgs, G.z_dim]).cuda()    # latent codes
    c = None                                # class labels (not used in this example)]
    w = G.mapping(z, c, truncation_psi=1, truncation_cutoff=8)
    img = G.synthesis(w, noise_mode='const', force_fp32=True)

    img = (img.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    x = transform(PIL.Image.fromarray(img.cpu().numpy(), 'RGB')).unsqueeze(0).to(device)
    with torch.no_grad():
      model.eval()
      logits = model(x)
      sigmoid_logits = torch.sigmoid(logits)
      predictions = (sigmoid_logits > 0.5).squeeze().cpu().numpy()
    target = set(labels[predictions.astype(bool)])
  w_noglasses.append(w)

In [ ]:
for i in range(400):
  img = G.synthesis(w_noglasses[i], noise_mode='const', force_fp32=True)
  img = (img.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  img_print = np.array(PIL .Image.fromarray(img.cpu().numpy(), 'RGB'))
  plt.ion()
  plt.figure()
  plt.imshow(img_print)
  plt.title(f'{i}')
  plt.show()
  _ = input('')
  plt.close()

Below we include the indexes of the handpicked images to save them as .npy files. If you choose to generate the images again, you'll have to handpick the images and change the indexes below.

In [ ]:
glasses_index = [0,5,6,7,8,9,10,15,20,21,23,24,26,28,31,36,38,40,41,42,48,51,52,53,54,56,58,59,
61,62,63,64,69,70,72,78,79,87,88,91,95,98,99,100,104,105,114,115,125,131,133,
139,144,150,151,153,156,162,175]

noglasses_index = [0,1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,20,25,26,27,28,29,31,32,
35,37,38,41,42,45,46,49,50,57,58,59,60,61,62,63,64,65,66,67,68,70,71,
72,73,74,75,76,77,78,79,80,82]

In [ ]:
w_glasses_list = np.array([x.cpu().numpy() for x in w_glasses])
w_glasses_list

In [ ]:
w_glasses_selected = w_glasses_list[glasses_index]

In [ ]:
w_noglasses_list = np.array([x.cpu().numpy() for x in w_noglasses])
w_noglasses_list

In [ ]:
w_noglasses_selected = w_noglasses_list[noglasses_index]

In [ ]:
np.save('/content/w_glasses_list.npy', np.array(w_glasses_list))
np.save('/content/w_noglasses_list.npy', np.array(w_noglasses_list))
np.save('/content/w_glasses_selected.npy', np.array(w_glasses_selected))
np.save('/content/w_noglasses_selected.npy', np.array(w_noglasses_selected))

In [ ]:
logits = model(x)
sigmoid_logits = torch.sigmoid(logits)
predictions = (sigmoid_logits > 0.5).squeeze().numpy()

print(labels[predictions.astype(bool)])

In [ ]:
img_testing = G.synthesis(w[35].unsqueeze(0), noise_mode='const', force_fp32=True)
img_testing = (img_testing.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
img_testing = np.array(PIL.Image.fromarray(img_testing.cpu().numpy(), 'RGB'))
plt.figure()
imshow(img_testing)

In [ ]:
sunglasses.append(w[11])
glasses.append(w[34])
hat.append(w[36])

## Load saved data (see above)

In [ ]:
def generate_img(G,w=None,z=None):
  c=None
  if(z!=None):
    w = G.mapping(z, c, truncation_psi=0.8, truncation_cutoff=8)
  # G is a Generator and w is the point in the mapping space (as a tensor)
  img = G.synthesis(w, noise_mode='const', force_fp32=True)
  img = (img.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  img = img.cpu().numpy()
  return img

def img_to_tensor(img):
  return transform(PIL.Image.fromarray(img, 'RGB')).unsqueeze(0).to(device)

## Moving in Z latent space



Below we download the saved Z values for the handpicked "glasses/no glasses" images and we use a SVM to find a separating hyperplane between these two glasses. Then, we do the same for the "young/old" classes.

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

### Glasses/no glasses

In [ ]:
!gdown --id 1VQl5ICbllJs1ePTGwndM3Y2QVNMMqaD0 -O /content/z_no_glasses_selected.npy
!gdown --id 1PQXKCVK7_HY5B9moVgDHhMPWsGR4lB5U -O /content/z_glasses_selected.npy

In [ ]:
z_noglasses_selected = torch.from_numpy(np.load("/content/z_no_glasses_selected.npy"))
z_glasses_selected = torch.from_numpy(np.load("/content/z_glasses_selected.npy"))

z_noglasses_selected = z_noglasses_selected.to(device)
z_glasses_selected = z_glasses_selected.to(device)

In [ ]:
img_glasses = [generate_img(G,z=z) for z in z_glasses_selected]
img_noglasses = [generate_img(G,z=z) for z in z_noglasses_selected]

# Transform tensors to numpy
z_noglasses_arr = np.squeeze(z_noglasses_selected.cpu().numpy())
z_glasses_arr = np.squeeze(z_glasses_selected.cpu().numpy())

# Space shape
space_shape = z_noglasses_arr[0].shape

In [ ]:
# Create X (points in mapping space) and y (classes)
X = np.concatenate((z_noglasses_arr, z_glasses_arr))
y = np.concatenate((np.zeros(z_noglasses_arr.shape[0]), np.ones(z_glasses_arr.shape[0])))

# Sort the samples
idx = random.sample(list(range(len(y))), len(y))
X = X[idx].reshape(X.shape[0], -1)
y = y[idx]

In [ ]:
# Fit SVM to find the separating hyperplane

clf_pipe = Pipeline([('scaler', StandardScaler()),
                          ('clf', SVC(gamma='auto', kernel='linear'))])
clf_pipe.fit(X, y)

In [ ]:
# Get hyperplane direction and normalize it

hyperplane_glasses = clf_pipe['clf'].coef_[0]
hyperplane_glasses = hyperplane_glasses.reshape(space_shape)
hyperplane_glasses = torch.from_numpy(hyperplane_glasses).to(device)
print(f'norm = {torch.norm(hyperplane_glasses)}')
hyperplane_glasses /= torch.norm(hyperplane_glasses)

Below, we use the hyperplane direction to move an image from a point in the "without glasses" side of the hyperplane to a point in the "with glasses" side of the hyperplane, moving in latent space Z, and we save these images as a gif.

In [ ]:
drive_path = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/saved_images/"
import imageio
images = []
img_n = 13
for i in range(30):
  z = z_noglasses_selected[img_n] + hyperplane_glasses*i
  img = generate_img(G, z=z)
  images.append(img)
imageio.mimsave(drive_path + f'moving_z_glasses_{img_n}.gif', images, duration=0.05)

#### Old/young

In [ ]:
!gdown --id 1Dlb8pv7hd-HEAmDkLBqar1DOT4QqTk3I -O /content/z_young_selected.npy
!gdown --id 1lzNq_YGGsjaEE2UlRCBNIM4Ygkksyr-H -O /content/z_old_selected.npy

In [ ]:
z_young_selected = torch.from_numpy(np.load("/content/z_young_selected.npy"))
z_old_selected = torch.from_numpy(np.load("/content/z_old_selected.npy"))

z_young_selected = z_young_selected.to(device)
z_old_selected = z_old_selected.to(device)

In [ ]:
img_young = [generate_img(G,z=z) for z in z_young_selected]
img_old = [generate_img(G,z=z) for z in z_old_selected]

# Transform tensors to numpy
z_young_arr = np.squeeze(z_young_selected.cpu().numpy())
z_old_arr = np.squeeze(z_old_selected.cpu().numpy())

# Space shape
space_shape = z_young_arr[0].shape

In [ ]:
# Create X (points in mapping space) and y (classes)
X = np.concatenate((z_young_arr, z_old_arr))
y = np.concatenate((np.zeros(z_young_arr.shape[0]), np.ones(z_old_arr.shape[0])))

# Sort the samples
idx = random.sample(list(range(len(y))), len(y))
X = X[idx].reshape(X.shape[0], -1)
y = y[idx]

In [ ]:
# Fit SVM to find the separating hyperplane

clf_pipe = Pipeline([('scaler', StandardScaler()),
                          ('clf', SVC(gamma='auto', kernel='linear'))])
clf_pipe.fit(X, y)

In [ ]:
# Get hyperplane direction and normalize it

hyperplane_age = clf_pipe['clf'].coef_[0]
hyperplane_age = hyperplane_age.reshape(space_shape)
hyperplane_age = torch.from_numpy(hyperplane_age).to(device)
print(f'norm = {torch.norm(hyperplane_age)}')
hyperplane_age /= torch.norm(hyperplane_age)

Below, we use the hyperplane direction to move an image from a point in the "young" side of the hyperplane to a point in the "old" side of the hyperplane, moving in latent space Z, and we save these images as a gif.

In [ ]:
drive_path = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/saved_images/"
import imageio
images = []
img_n = 13
for i in range(30):
  z = z_young_selected[img_n] + hyperplane_age*i
  img = generate_img(G, z=z)
  images.append(img)
imageio.mimsave(drive_path + f'moving_z_age_{img_n}.gif', images, duration=0.05)

### Conditioning in Z space

Now we'll use the method proposed in the article as "Conditional manipulation via subspace projection". After finding the hyperplanes corresponding to two different features, we project one plane into the other and subtract this projection from the first plane to find a direction perpendicular to the second one. In that way, when we move in this direction we're not supposed to change the other feature.

In [ ]:
hyperplane_glasses_conditioned = hyperplane_glasses-torch.matmul(hyperplane_glasses.T,hyperplane_age)*hyperplane_age

In [ ]:
torch.matmul(hyperplane_glasses_conditioned,hyperplane_glasses)

In [ ]:
torch.matmul(hyperplane_glasses_conditioned,hyperplane_age)

In [ ]:
drive_path = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/saved_images/"
import imageio
images = []
img_n = 13
for i in range(30):
  z = z_noglasses_selected[img_n] + hyperplane_glasses_conditioned*i
  img = generate_img(G, z=z)
  images.append(img)
imageio.mimsave(drive_path + f'moving_z_conditioned_{img_n}.gif', images, duration=0.05)

## Using hundreds of images for getting the SVM:

As an alternative to handpicking images which fit our features criteria, we've tried using the SlimNet classifier to classify a thousand of images from each class studied: with glasses, without glasses, young, old, female and male. We found separating hyperplanes as before and saved the gifs showing the result from the transition between two points in latent space. The results with SlimNet classified samples was not that good, especially concerning the "glasses" case of study. Checking manually, we found that the accuracy of the "Eyeglasses" label on this dataset was lesser than 50%, i.e., many "no glasses" images were labeled as "glasses".

### Glasses / No glasses

In [ ]:
# Generating images with/without glasses

w_glasses = []
w_no_glasses = []
i_g = 0
i_ng = 0
n_imgs = 1000
idx = list(labels).index("Eyeglasses")

while i_g < n_imgs or i_ng < n_imgs:
  z = torch.randn([1, G.z_dim]).cuda()    # latent codes
  c = None                                # class labels (not used in this example)]
  w = G.mapping(z, c, truncation_psi=1, truncation_cutoff=8)
  img = G.synthesis(w, noise_mode='const', force_fp32=True)

  img = (img.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  x = transform(PIL.Image.fromarray(img.cpu().numpy(), 'RGB')).unsqueeze(0).to(device)
  with torch.no_grad():
    model.eval()
    logits = model(x)
    sigmoid_logits = torch.sigmoid(logits)
    predictions = (sigmoid_logits > 0.5).squeeze().cpu().numpy()
  
  x_labels = set(labels[predictions.astype(bool)])
  if "Eyeglasses" in x_labels:
    if i_g < n_imgs:
      w_glasses.append(w)
      i_g += 1
  else:
    if i_ng < n_imgs:
      w_no_glasses.append(w)
      i_ng += 1

In [ ]:
# Transform tensors to numpy
w_no_glasses_arr = [w.cpu().numpy().flatten() for w in w_no_glasses]
w_glasses_arr = [w.cpu().numpy().flatten() for w in w_glasses]

# Space shape
space_shape = w_glasses_arr[0].shape

# Create X (points in mapping space) and y (classes)
X = np.concatenate((w_no_glasses_arr, w_glasses_arr))
y = np.concatenate((np.zeros(n_imgs), np.ones(n_imgs)))

# Sort the samples
idx = random.sample(list(range(len(y))), len(y))
X = X[idx].reshape(X.shape[0], -1)
y = y[idx]

# Getting SVM hyperplane:

clf_pipe = Pipeline([('scaler', StandardScaler()),
                          ('clf', SVC(gamma='auto', kernel='linear'))])
clf_pipe.fit(X, y)

In [ ]:
# Generate GIF with one of the images:
orig_shape = [1, 14, 512]

hyperplane_glasses = clf_pipe['clf'].coef_[0]
hyperplane_glasses = hyperplane_glasses.reshape(orig_shape)
hyperplane_glasses = torch.from_numpy(hyperplane_glasses).to(device)
print(f'norm = {torch.norm(hyperplane_glasses)}')
hyperplane_glasses /= torch.norm(hyperplane_glasses)

drive_path = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/saved_images/"
import imageio
images = []
img_n = 151
for i in range(100):
  w = w_no_glasses[img_n] + hyperplane_glasses*i
  img = generate_img(G, w)
  images.append(img)
imageio.mimsave(drive_path + f'SVM_1000_glasses_{img_n}.gif', images, duration=0.1)

### Old / Young

In [ ]:
# Generating images with/without glasses

w_young = []
w_old = []
i_g = 0
i_ng = 0
n_imgs = 1000

while i_g < n_imgs or i_ng < n_imgs:
  z = torch.randn([1, G.z_dim]).cuda()    # latent codes
  c = None                                # class labels (not used in this example)]
  w = G.mapping(z, c, truncation_psi=1, truncation_cutoff=8)
  img = G.synthesis(w, noise_mode='const', force_fp32=True)

  img = (img.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  x = transform(PIL.Image.fromarray(img.cpu().numpy(), 'RGB')).unsqueeze(0).to(device)
  with torch.no_grad():
    model.eval()
    logits = model(x)
    sigmoid_logits = torch.sigmoid(logits)
    predictions = (sigmoid_logits > 0.5).squeeze().cpu().numpy()
  
  x_labels = set(labels[predictions.astype(bool)])
  if "Young" in x_labels:
    if i_g < n_imgs:
      w_young.append(w)
      i_g += 1
  else:
    if i_ng < n_imgs:
      w_old.append(w)
      i_ng += 1

In [ ]:
# Transform tensors to numpy
w_old_arr = [w.cpu().numpy().flatten() for w in w_old]
w_young_arr = [w.cpu().numpy().flatten() for w in w_young]

# Space shape
space_shape = w_young_arr[0].shape

# Create X (points in mapping space) and y (classes)
X = np.concatenate((w_old_arr, w_young_arr))
y = np.concatenate((np.zeros(n_imgs), np.ones(n_imgs)))

# Sort the samples
idx = random.sample(list(range(len(y))), len(y))
X = X[idx].reshape(X.shape[0], -1)
y = y[idx]

# Getting SVM hyperplane:

clf_pipe = Pipeline([('scaler', StandardScaler()),
                          ('clf', SVC(gamma='auto', kernel='linear'))])
clf_pipe.fit(X, y)

In [ ]:
# Generate GIF with one of the images:
orig_shape = [1, 14, 512]

hyperplane_age = clf_pipe['clf'].coef_[0]
hyperplane_age = hyperplane_age.reshape(orig_shape)
hyperplane_age = torch.from_numpy(hyperplane_age).to(device)
print(f'norm = {torch.norm(hyperplane_age)}')
hyperplane_age /= torch.norm(hyperplane_age)

drive_path = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/saved_images/"
import imageio
images = []
img_n = 577
for i in range(150):
  w = w_young[img_n] + hyperplane_age*i
  img = generate_img(G, w)
  images.append(img)
imageio.mimsave(drive_path + f'SVM_1000_age_{img_n}.gif', images, duration=0.05)

## Quantitative evaluation

We keep the code below for documentation purposes, but we were not able to test the evaluation metrics defined in the article. Both of the metrics proposed (path length and linear separability) need the full dataset saved as Multiresolution TFRecords. Moreover, the "linear separability" metric isn't available for the official PyTorch implementation of the Stylegan2 ADA, which's the source code we're using.

In [ ]:
# Generate image for each selected point

img_young = [generate_img(G,w) for w in w_young_selected]
img_old = [generate_img(G,w) for w in w_old_selected]

# Transform tensors to numpy
w_young_arr = np.squeeze(w_young_selected.cpu().numpy())
w_old_arr = np.squeeze(w_old_selected.cpu().numpy())

# Space shape
space_shape = w_young_arr[0].shape

In [ ]:
w = w_young_selected[0]

img = G.synthesis(w, noise_mode='const', force_fp32=True)
img = (img.squeeze(0).permute(1, 2, 0) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
img = img.cpu().numpy()


In [ ]:
!gdown --id 1Dlb8pv7hd-HEAmDkLBqar1DOT4QqTk3I -O /content/z_young_selected.npy

z_young_selected = torch.from_numpy(np.load("/content/z_young_selected.npy"))
z = z_young_selected[0].cuda()
w = G.mapping(z, c, truncation_psi=1, truncation_cutoff=8)


In [ ]:
drive_path = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/saved_images/"

In [ ]:
!python calc_metrics.py --metrics=ppl2_wend --network=/content/celebahq-res256-mirror-paper256-kimg100000-ada-target0.5.pkl

In [ ]:
#!git clone https://github.com/NVlabs/stylegan